In [4]:
import os
import matplotlib.pyplot as plt
%matplotlib inline

import torch
from torch.utils import data
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models

from torchvision import datasets

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from models import get_model, get_pre_loaded_model
from training import compute_accuracy, train
from tools import save_net, show_predictions

In [8]:
skip_training = False
pre_trained = False

In [6]:
device = torch.device('cuda:0')

In [13]:
transform = transforms.Compose([
    transforms.ToTensor(),  # Transform to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
])

data_dir = 'tiny-imagenet-200/'
train_dataset = datasets.ImageFolder(os.path.join(data_dir, 'train'), transform)
test_dataset = datasets.ImageFolder(os.path.join(data_dir, 'test'), transform)
val_dataset = datasets.ImageFolder(os.path.join(data_dir, 'val'), transform)

trainloader = data.DataLoader(train_dataset, batch_size=50, shuffle=True)
valloader = data.DataLoader(val_dataset, batch_size=5, shuffle=True)
testloader = data.DataLoader(test_dataset, batch_size=5, shuffle=True)

# Quick search
1. [Resnet](#resnet)
2. [SqueezeNet](#squeeze)
3. [AlexNet](#alexnet)

## ResNet <a name="resnet"></a>

In [14]:
if skip_training:
    net = get_pre_loaded_model('resnet','models/pretrained_resnet_epoch_50.pth')
else:
    net = get_model('resnet', pre_trained)
net.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [15]:
# Define the loss and the optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [16]:
n_epochs = 50

In [17]:
if not skip_training:
    train(net,device,n_epochs, optimizer,criterion,trainloader,valloader)
else:
    accuracy = compute_accuracy(net, device, valloader)
    print('Accuracy of the network on the test images: %.3f' % accuracy)

[1,   200] loss: 6.232
[1,   400] loss: 5.440
[1,   600] loss: 5.170
[1,   800] loss: 5.037
[1,  1000] loss: 4.926
[1,  1200] loss: 4.838
[1,  1400] loss: 4.744
[1,  1600] loss: 4.676
[1,  1800] loss: 4.572
[1,  2000] loss: 4.492
Accuracy of the network on the test images: 0.079
[2,   200] loss: 4.442
[2,   400] loss: 4.399
[2,   600] loss: 4.306


KeyboardInterrupt: 

In [0]:
if not skip_training:
    filename = 'resnet_epoch_50.pth'
    save_net(net, filename)

In [62]:
# Show the ground truth labels and the network's predictions
show_predictions(net, device, testloader)

Prediction 1:
Turth: cash machine, cash dispenser, automated teller machine, automatic teller machine, automated teller, automatic teller, ATM
Pred: European fire salamander, Salamandra salamandra

Prediction 2:
Turth: ice cream, icecream
Pred: ice cream, icecream

Prediction 3:
Turth: water jug
Pred: moving van

Prediction 4:
Turth: kimono
Pred: birdhouse

Prediction 5:
Turth: comic book
Pred: cougar, puma, catamount, mountain lion, painter, panther, Felis concolor



Let us now compute the accuracy of the network on the test set.

In [63]:
# Accuracy on the test set
accuracy = compute_accuracy(net, device, testloader)
print('Accuracy of the network on the test images: %.3f' % accuracy)

Accuracy of the network on the test images: 0.658


In [0]:
if torch.cuda.is_available():
    torch.cuda.empty_cache()

## SqueezeNet  <a name="squeeze"></a>

In [18]:
if skip_training:
    net = get_pre_loaded_model('squeeze','models/pretrained_squeezenet_epoch_50.pth')
else:
    net = get_model('squeeze',pre_trained)
net.to(device)

SqueezeNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (3): Fire(
      (squeeze): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace)
    )
    (4): Fire(
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace)
    )
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0

In [19]:
# Define the loss and the optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [20]:
n_epochs = 50

In [21]:
if not skip_training:
    train(net,device,n_epochs, optimizer,criterion,trainloader,valloader)
else:
    accuracy = compute_accuracy(net, device, valloader)
    print('Accuracy of the network on the test images: %.3f' % accuracy)

[1,   200] loss: 5.308
[1,   400] loss: 5.298
[1,   600] loss: 5.298
[1,   800] loss: 5.296
[1,  1000] loss: 5.295
[1,  1200] loss: 5.290
[1,  1400] loss: 5.265
[1,  1600] loss: 5.232


KeyboardInterrupt: 

In [0]:
if not skip_training:
    filename = 'resnet_epoch_50.pth'
    save_net(net, filename)

In [75]:
# Show the ground truth labels and the network's predictions
show_predictions(net, device, testloader)

Prediction 1:
Turth: candle, taper, wax light
Pred: thatch, thatched roof

Prediction 2:
Turth: bell pepper
Pred: bell pepper

Prediction 3:
Turth: chain
Pred: chain

Prediction 4:
Turth: basketball
Pred: basketball

Prediction 5:
Turth: scoreboard
Pred: scoreboard



Let us now compute the accuracy of the network on the test set.

In [77]:
# Accuracy on the test set
accuracy = compute_accuracy(net, device, testloader)
print('Accuracy of the network on the test images: %.3f' % accuracy)

Accuracy of the network on the test images: 0.539


In [0]:
if torch.cuda.is_available():
    torch.cuda.empty_cache()

## AlexNet <a name="alexnet"></a>

In [22]:
if skip_training:
    net = get_pre_loaded_model('alexnet','models/pertrained_alexnet_epoch_50.pth')
else:
    net = get_model('alexnet',pre_trained)
net.to(device)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
    (2): ReLU(inplace)
    (3): Dropout(p

In [23]:
# Define the loss and the optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [24]:
n_epochs = 50

In [25]:
if not skip_training:
    train(net,device,n_epochs, optimizer,criterion,trainloader,valloader)
else:
    accuracy = compute_accuracy(net, device, valloader)
    print('Accuracy of the network on the test images: %.3f' % accuracy)

[1,   200] loss: 6.901
[1,   400] loss: 6.631


KeyboardInterrupt: 

In [0]:
if not skip_training:
    filename = 'resnet_epoch_50.pth'
    save_net(net, filename)

In [87]:
# Show the ground truth labels and the network's predictions
show_predictions(net, device, testloader)

Prediction 1:
Turth: acorn
Pred: standard poodle

Prediction 2:
Turth: pill bottle
Pred: pill bottle

Prediction 3:
Turth: syringe
Pred: syringe

Prediction 4:
Turth: teapot
Pred: teapot

Prediction 5:
Turth: water tower
Pred: cliff, drop, drop-off



Let us now compute the accuracy of the network on the test set.

In [88]:
# Accuracy on the test set
accuracy = compute_accuracy(net, device, testloader)
print('Accuracy of the network on the test images: %.3f' % accuracy)

Accuracy of the network on the test images: 0.562


In [0]:
if torch.cuda.is_available():
    torch.cuda.empty_cache()